In [1]:
import ee
import pandas as pd
from datetime import datetime, timedelta, timezone
import pytz
from scipy.spatial import cKDTree

# Initialize the Earth Engine module with the project ID
ee.Initialize(project='data690-zhouhaomatt')

# Define the points of interest
points_of_interest = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

# Convert points to EE objects
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

# Define the ROI using a central point and buffer
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

# Setup image and cloud score collections
band_list = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
cloud_bands = ['cs', 'cs_cdf']

images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2018-01-01', '2024-06-19')\
    .filterBounds(roi)

clouds = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')\
    .filterDate('2018-01-01', '2024-06-19')\
    .filterBounds(roi)

# Data extraction setup
region = ee.Geometry.MultiPoint(ee_points)
scale = 10

def convert_df(data, timezone='Etc/GMT-12'):
    df = pd.DataFrame(data[1:], columns=data[0])
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert(timezone)
    df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

def find_nearest_points(gee_points, poi_points):
    tree = cKDTree(poi_points)
    distances, indices = tree.query(gee_points)
    return indices

# Extract data
try:
    pixel_values = images.select(band_list).getRegion(region, scale).getInfo()
    cloud_values = clouds.select(cloud_bands).getRegion(region, scale).getInfo()
except Exception as e:
    print(f"Failed to retrieve data: {e}")
    exit()

# Convert to pandas dataframes
pixel_values_df = convert_df(pixel_values)
cloud_values_df = convert_df(cloud_values)

# Extract solar angles from image metadata
def get_solar_angles(images):
    solar_data = []
    for image in images.toList(images.size()).getInfo():
        solar_zenith = image['properties'].get('MEAN_SOLAR_ZENITH_ANGLE', None)
        solar_azimuth = image['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE', None)
        timestamp = image['properties']['system:time_start']
        solar_data.append({
            'timestamp': datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            'solar_zenith': solar_zenith,
            'solar_azimuth': solar_azimuth
        })
    return pd.DataFrame(solar_data)

solar_df = get_solar_angles(images)

# Get unique GEE returned points
gee_points = list(zip(pixel_values_df['longitude'].unique(), pixel_values_df['latitude'].unique()))

# Find nearest POIs for each GEE returned point
poi_indices = find_nearest_points(gee_points, points_of_interest)

# Map GEE points to POIs
pixel_values_df['poi_index'] = find_nearest_points(list(zip(pixel_values_df['longitude'], pixel_values_df['latitude'])), points_of_interest)
cloud_values_df['poi_index'] = find_nearest_points(list(zip(cloud_values_df['longitude'], cloud_values_df['latitude'])), points_of_interest)

# Merge DataFrames based on Nearest POI
def merge_dataframes_by_poi(pixels_df, clouds_df):
    merged_dfs = []
    for i in range(len(points_of_interest)):
        point_pixels = pixels_df[pixels_df['poi_index'] == i].drop(columns=['longitude', 'latitude', 'poi_index'])
        point_clouds = clouds_df[clouds_df['poi_index'] == i].drop(columns=['longitude', 'latitude', 'poi_index'])
        
        point_df = pd.merge(point_pixels, point_clouds, on='timestamp', how='outer', suffixes=('', '_cloud'))
        point_df = point_df.rename(columns=lambda x: f'point_{i+1}_{x}' if x not in ['timestamp'] else x)
        merged_dfs.append(point_df)
    
    # Concatenate all point DataFrames
    merged_df = pd.concat(merged_dfs, axis=1)
    return merged_df

# Example usage
final_df = merge_dataframes_by_poi(pixel_values_df, cloud_values_df)

# Combine solar angles with the final DataFrame
final_df = final_df.merge(solar_df, on='timestamp', how='left')

print(final_df.head())


/var/folders/q3/x0b5k_7d55ld8ywy_cs0vhy80000gn/T/ipykernel_34685/1157829247.py:77: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'timestamp': datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S'),


ValueError: The column label 'timestamp' is not unique.